In [ ]:
from pyspark.sql.functions import lit

In [ ]:
# file to be loaded in landing zone and current processing date
# values in these variable will be dynamically added using parameter
today_file = 'LMS_09-01-2023.csv'  # LMS_09-01-2023.csv
processing_date = '2025-12-29' # 2025-12-29
storage_account_name = 'delakehouse'
container_name = 'lmsproject'
raw_folder = 'raw'
landing_folder = 'landing'
destination_account_name = 'delakehouse'
destination_container_name = 'lmsproject'

In [ ]:
# abfss path src for azure container
abfs_path_src = f'abfss://{container_name}@{storage_account_name}.dfs.core.windows.net'
# abfss path destination for azure container
abfs_path_dest = f'abfss://{destination_container_name}@{destination_account_name}.dfs.core.windows.net'

In [ ]:
raw_file_path = f'{abfs_path_src}/{raw_folder}/{today_file}'
landing_file_path = f'{abfs_path_dest}/{landing_folder}/'
try:
    df = (
        spark.read.format("csv")
        .option("inferSchema",True)
        .option("header",True)
        .load(path=raw_file_path)
    )
    if df.count() > 1:
        print("File contains data")
        df_with_column = df.withColumn("Processing_Date",lit(processing_date))
        df_with_column.write.format("csv").option("header",True).mode("append").partitionBy("Processing_Date").save(landing_file_path)
        print("Data written to landing folder") 
    else:
        print("File does not contain data")
except Exception as e:
    print(e)